In [2]:
### 🚀 RUN ME BEFORE YOU START WORKING ON THIS NOTEBOOK ⚠️

import helpers
NOTEBOOK_PATH = __vsc_ipynb_file__ # type: ignore
helpers.initialize(notebook_path=NOTEBOOK_PATH)

### 🚀 RUN ME BEFORE YOU START WORKING ON THIS NOTEBOOK ⚠️

✅ Added notebook directory to Python path:
   /workspaces/llmops-course/modules/model_registry/solutions
   You can now import modules from this directory
⚠️ Warning: Failed to set up notebook path
🔄 Initializing Course environment...
🔁 Autoreload enabled: modules will reload automatically when changed
🔕 Suppressed future deprecation warnings
📝 Logging configured
📊 Pandas display settings configured for better output
🔍 Looking for .env file at: /workspaces/llmops-course/.env
✅ Successfully loaded environment variables from /workspaces/llmops-course/.env
📋 Loaded variables: GEMINI_API_KEY=****sbqo
⚙️ Disabled MLflow system metrics logging
📔 Disabled MLflow notebook display (avoids VSCode bugs)

🎉 All systems go! Your Course environment is ready for learning!


In [30]:
MODEL_NAME = "article_summarizer"


In [31]:
import mlflow
from mlflow.exceptions import RestException

client = mlflow.tracking.MlflowClient()
try:
    client.delete_registered_model(name=MODEL_NAME)
    print("✨ Model deleted and we are ready to go! 🚀")
except RestException as e:
    if "RESOURCE_DOES_NOT_EXIST" in str(e):
        print("✨ Model not found, we are ready to go! 🚀") 


✨ Model not found, we are ready to go! 🚀


In [32]:
import mlflow
from textsummarizer import SerializableArticleSummarizerModel
import pandas as pd
import os

path = 'assets/articles_full_length/ai_relationships.html'
text = open(path).read()

input_example = pd.DataFrame({'text': [text]})

m = SerializableArticleSummarizerModel()

with mlflow.start_run(run_name="Article Summarizer") as run:
     mlflow.pyfunc.log_model(
        MODEL_NAME,
        python_model=m,
        input_example=input_example,
        signature=mlflow.models.infer_signature(
            input_example,
            m.predict(input_example)
        ),

        code_paths=[os.path.dirname(__vsc_ipynb_file__)],
        registered_model_name="article_summarizer",
    )


2025/03/04 08:54:19 INFO mlflow.pyfunc: Validating input example against model signature
2025/03/04 08:54:26 INFO mlflow.models.model: Found the following environment variables used during model inference: [GEMINI_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
Successfully registered model 'article_summarizer'.
2025/03/04 08:54:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: article_summarizer, version 1


🏃 View run Article Summarizer at: http://localhost:5050/#/experiments/0/runs/8d24eeb4572d42479e6eb7c248578869
🧪 View experiment at: http://localhost:5050/#/experiments/0


Created version '1' of model 'article_summarizer'.


In [33]:
client = mlflow.tracking.MlflowClient()
model_version = client.get_latest_versions(
    name=MODEL_NAME
)[0]

print(f"Model: {model_version.name},  Version: {model_version.version}, Aliases: {model_version.aliases}")
model_uri = f"models://{model_version.name}/{model_version.version}"
print(f"Model URI: {model_uri}")

Model: article_summarizer,  Version: 1, Aliases: []
Model URI: models://article_summarizer/1


In [34]:

# Set the alias to "production" instead of using stages
client.set_registered_model_alias(
    name=model_version.name,
    alias="test",
    version=model_version.version
)

print(f"Model {model_version.name} version {model_version.version} set with 'test' alias")


Model article_summarizer version 1 set with 'test' alias


In [36]:
model_version = client.get_model_version_by_alias(
    name=MODEL_NAME,
    alias="test"
)

print(f"Model: {model_version.name},  Version: {model_version.version}, Aliases: {model_version.aliases}")
model_uri = f"models://{model_version.name}/{model_version.version}"
print(f"Model URI: {model_uri}")


Model: article_summarizer,  Version: 1, Aliases: ['test']
Model URI: models://article_summarizer/1


Improve our model

In [37]:
# Remove test alias and set production alias
client.delete_registered_model_alias(
    name=MODEL_NAME,
    alias="test"
)

client.set_registered_model_alias(
    name=MODEL_NAME,
    alias="production",
    version=model_version.version
)

print(f"Model {MODEL_NAME} version {model_version.version} promoted to production")


Model article_summarizer version 1 promoted to production


Let's improve our model

In [39]:
import mlflow
from textsummarizer import HTMLSummarizerModel
import pandas as pd
import os

path = 'assets/articles_full_length/ai_relationships.html'
text = open(path).read()

input_example = pd.DataFrame({'text': [text]})

m = HTMLSummarizerModel()


with mlflow.start_run(run_name="Article Summarizer") as run:
    # ... do all the tests
    model_info = mlflow.pyfunc.log_model(
        MODEL_NAME,
        python_model=m,
        input_example=input_example,
        signature=mlflow.models.infer_signature(
            input_example,
            m.predict(input_example)
        ),

        code_paths=[os.path.dirname(__vsc_ipynb_file__)],
    )


2025/03/04 09:01:12 INFO mlflow.pyfunc: Validating input example against model signature
2025/03/04 09:01:18 INFO mlflow.models.model: Found the following environment variables used during model inference: [GEMINI_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run Article Summarizer at: http://localhost:5050/#/experiments/0/runs/75107e8c68ba49878a04a1103cd3d2b2
🧪 View experiment at: http://localhost:5050/#/experiments/0


In [44]:
# Register the model version
model_version = mlflow.register_model(
    model_info.model_uri,
    MODEL_NAME
)

print(f"Run-based model URI: {model_info.model_uri}")
print(f"Registry-based models URI: models://{MODEL_NAME}/{model_version.version}")

Registered model 'article_summarizer' already exists. Creating a new version of this model...
2025/03/04 09:02:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: article_summarizer, version 6


Run-based model URI: runs:/75107e8c68ba49878a04a1103cd3d2b2/article_summarizer
Registry-based models URI: models://article_summarizer/6


Created version '6' of model 'article_summarizer'.


In [45]:
# Promote the model to test
client = mlflow.MlflowClient()
client.set_registered_model_alias(MODEL_NAME, "test", model_version.version)

# Validate the alias was set correctly
test_version = client.get_model_version_by_alias(MODEL_NAME, "test")
print(f"Model version {test_version.version} is now marked as 'test'")



Model version 6 is now marked as 'test'


In [48]:
# Archive the current production model if it exists
try:
    current_prod = client.get_model_version_by_alias(MODEL_NAME, "production")
    client.set_registered_model_alias(MODEL_NAME, "archived", current_prod.version)
    print(f"Archived previous production model version {current_prod.version}")
except:
    print("No previous production model found to archive")

# Promote test to production
client.set_registered_model_alias(MODEL_NAME, "production", test_version.version)
print(f"Model version {test_version.version} is now marked as 'production'")


Archived previous production model version 6
Model version 6 is now marked as 'production'
